In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [8]:
df = pd.read_csv('../data/historical/Batting.csv')

In [9]:
df.groupby('yearID').sum()

,stint,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
yearID,,,,,,,,,,,,,,,,,,
1871,115,2296,10822,2659,3101,434,239,47,1783.0,441.0,123.0,393,175.0,0.0,0.0,0.0,0.0,74.0
1872,172,3305,15663,3390,4467,581,145,37,2132.0,269.0,134.0,263,264.0,0.0,0.0,0.0,0.0,97.0
1873,128,3604,16959,3580,4926,570,211,47,2331.0,314.0,131.0,335,278.0,0.0,0.0,0.0,0.0,122.0
1874,126,4199,19104,3470,5224,634,194,40,2252.0,242.0,97.0,238,357.0,0.0,0.0,0.0,0.0,107.0
1875,248,6248,26833,4234,6812,839,273,40,2710.0,629.0,320.0,249,675.0,0.0,0.0,0.0,0.0,142.0
1876,127,4696,20121,3066,5338,633,181,40,1984.0,0.0,0.0,336,589.0,0.0,0.0,0.0,0.0,0.0
1877,106,3247,13667,2040,3705,431,204,24,1410.0,0.0,0.0,345,726.0,0.0,0.0,0.0,0.0,0.0
1878,83,3319,13644,1904,3539,481,132,23,1331.0,0.0,0.0,364,1081.0,0.0,0.0,0.0,0.0,0.0
1879,135,5795,24155,3409,6171,958,317,58,2357.0,0.0,0.0,508,1843.0,0.0,0.0,0.0,0.0,0.0
